In [ ]:
def myFunc(x): return x

In [2]:
def strat_k_fold(annotations_file, n_splits=5, random_state=None, shuffle=True): #sampling_method=r_os):
    """Applies stratified kfold based on/integrated from scikit-learn:
    https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
    """
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import StratifiedKFold
    
    if isinstance(annotations_file, pd.DataFrame):
        
        annotations_file
        
    else:
        annotations_file = pd.read_excel(annotations_file) 
    
    df_kfold = annotations_file.copy()
    X = np.array(annotations_file.iloc[:,0]).reshape(-1,1)
    y = np.array(annotations_file.iloc[:,1]).reshape(-1,1)
    
    skf = StratifiedKFold(n_splits=n_splits,
    shuffle=shuffle, random_state=random_state)

    i = 0
    
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        df_kfold.loc[train_index, f'split{i}'] = 'False'
        df_kfold.loc[test_index, f'split{i}'] = 'True'
        i += 1
        
    return df_kfold

In [1]:
def sampler_strat_kfold(df_strat_kfold, rs='ros', random_state=None, random_state_valid=None):
    """For validation sets, random-oversampling is always used to make random-oversampling and
    random-undersampling more comparable. "rs": specifies sampler for training."""
    
    import numpy as np
    import pandas as pd
    from imblearn.under_sampling import RandomUnderSampler
    from imblearn.over_sampling import RandomOverSampler 
    
    n_splits = (len(df_strat_kfold.columns)-2)

    dfs_compl = []

    
    rs = rs
    
    if rs == 'ros':
        rs = RandomOverSampler(random_state=random_state)
        
    elif rs == 'rus':
        rs = RandomUnderSampler(random_state=random_state)
        
    rs_valid = RandomOverSampler(random_state=random_state_valid)

    for i in range(n_splits):

        df = df_strat_kfold.iloc[:,[0,1,(i+2)]]

        df_train = df[df[df.columns[2]] == 'False'].reset_index(drop=True)
        df_valid = df[df[df.columns[2]] == 'True'].reset_index(drop=True)

        X = np.array(df_train[df_train.columns[0]]).reshape(-1,1)
        y = np.array(df_train[df_train.columns[1]]).reshape(-1,1)

        X_res_train, y_res_train = rs.fit_resample(X, y)

        df_train = df_train[:0]

        df_train[df_train.columns[0]] = [i.item() for i in X_res_train]
        df_train[df_train.columns[1]] = y_res_train.tolist()
        df_train[df_train.columns[2]] = False

        X = np.array(df_valid[df_valid.columns[0]]).reshape(-1,1)
        y = np.array(df_valid[df_valid.columns[1]]).reshape(-1,1)

        X_res_valid, y_res_valid = rs_valid.fit_resample(X, y)

        df_valid = df_valid[:0]

        df_valid[df_valid.columns[0]] = [i.item() for i in X_res_valid]
        df_valid[df_valid.columns[1]] = y_res_valid.tolist()
        df_valid[df_valid.columns[2]] = True

        #df_compl = df_train.append(df_valid).reset_index(drop=True)
        df_compl = pd.concat([df_train, df_valid]).reset_index(drop=True)
        dfs_compl.append(df_compl)
    
    return dfs_compl
    

In [1]:
def add_imgs_dataset(dfs, dir_cases, tiles_case=50, random_state=None):
    import os
    import pandas as pd
    import random
    import numpy as np
    
    import re
    random.seed(random_state)
    
    dfs_new = []
    dict_imgs = {i[0][:12] if i[0][:4] == "TCGA" else re.match("^.*?(?=\/)", i[0]).group(0)[:-5]: i for i in [[i+'/'+s for s in os.listdir(dir_cases+i)] for i in [i for i in os.listdir(dir_cases)]]}
    for k in range(len(dfs)):
        df_new = pd.DataFrame({dfs[0].columns[0]: np.hstack([random.sample(dict_imgs.get(i),tiles_case) if tiles_case <= len(dict_imgs.get(i)) else random.choices(dict_imgs.get(i),k=tiles_case,) for i in dfs[k][dfs[0].columns[0]]]).tolist(),
               dfs[0].columns[1] : np.hstack([[dfs[k].iloc[i, 1] for x in range(tiles_case)] for i in range(len(dfs[k]))]).tolist(),
               f'split{k}' : np.hstack([[dfs[k].iloc[i, 2] for x in range(tiles_case)] for i in range(len(dfs[k]))]).tolist()
              })
        dfs_new.append(df_new)
        
    return dfs_new

In [ ]:
def getListOfFiles(dirName):
    import os
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles